# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv('../../datasets/day-of-week-not-scaled.csv')
df_scaled = pd.read_csv('../../datasets/dayofweek.csv')
df['dayofweek'] = df_scaled['dayofweek']
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [4]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [5]:
model = SVC(random_state=21, probability=True)
param_grid = {
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ['balanced', None]
}

gird_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
gird_search.fit(X_train, y_train)

print(gird_search.best_params_)
print(gird_search.best_score_)

{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
0.8761090458488228


In [6]:
results_df = pd.DataFrame(gird_search.cv_results_)
results_df.sort_values(by='rank_test_score', ascending=True)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.253084,0.018820,0.007962,0.001483,0.01,balanced,scale,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
1,0.393148,0.029852,0.026816,0.003424,0.01,balanced,scale,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
2,0.373669,0.029651,0.015129,0.005106,0.01,balanced,scale,sigmoid,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
3,0.292808,0.044766,0.008863,0.002226,0.01,balanced,auto,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
4,0.402823,0.023764,0.028486,0.003415,0.01,balanced,auto,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [7]:
model = DecisionTreeClassifier(random_state=21)
param_grid = {
    'max_depth': list(range(1,49)),
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 23}
0.8738592867960897


In [8]:
results_df = pd.DataFrame(gird_search.cv_results_)
results_df.sort_values(by='rank_test_score', ascending=True)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.253084,0.018820,0.007962,0.001483,0.01,balanced,scale,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
1,0.393148,0.029852,0.026816,0.003424,0.01,balanced,scale,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
2,0.373669,0.029651,0.015129,0.005106,0.01,balanced,scale,sigmoid,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
3,0.292808,0.044766,0.008863,0.002226,0.01,balanced,auto,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
4,0.402823,0.023764,0.028486,0.003415,0.01,balanced,auto,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [9]:
model = RandomForestClassifier(random_state=21)

params_grid = {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': list(range(1, 49)),
    'class_weight': ['balanced', None],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(model, params_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50}
0.9042902381935839


In [10]:
results_df = pd.DataFrame(gird_search.cv_results_)
results_df.sort_values(by='rank_test_score', ascending=True)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.253084,0.018820,0.007962,0.001483,0.01,balanced,scale,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
1,0.393148,0.029852,0.026816,0.003424,0.01,balanced,scale,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
2,0.373669,0.029651,0.015129,0.005106,0.01,balanced,scale,sigmoid,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59
3,0.292808,0.044766,0.008863,0.002226,0.01,balanced,auto,linear,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.325926,0.344444,0.337037,0.345725,0.345725,0.339771,0.007641,36
4,0.402823,0.023764,0.028486,0.003415,0.01,balanced,auto,rbf,"{'C': 0.01, 'class_weight': 'balanced', 'gamma...",0.233333,0.233333,0.233333,0.059480,0.237918,0.199480,0.070023,59


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [13]:
params_grid = {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': list(range(1, 49)),
    'class_weight': ['balanced', None],
    'criterion': ['gini', 'entropy']
}

results = []

param_combinations = list(product(params_grid['n_estimators'], params_grid['max_depth'], params_grid['class_weight'], params_grid['criterion']))
 
for n, max_depth_val, class_weight_val, criterion_val in tqdm(param_combinations):
    model = RandomForestClassifier(n_estimators=n, max_depth=max_depth_val, class_weight=class_weight_val, criterion=criterion_val)
    scores = cross_val_score(model, X_train, y_train, cv=5)

    results.append({
        'n_estimators': n,
        'max_depth': max_depth_val,
        'class_weight': class_weight_val,
        'criterion': criterion_val,
        'mean_score': scores.mean(),
        'std_score': scores.std()
    })

  0%|          | 0/768 [00:00<?, ?it/s]

In [19]:
results_df = pd.DataFrame(results)
results_df.sort_values(by='mean_score', ascending=False)
results_df.head()

,n_estimators,max_depth,class_weight,criterion,mean_score,std_score
0,5,1,balanced,gini,0.281159,0.071824
1,5,1,balanced,entropy,0.272169,0.051312
2,5,1,None,gini,0.351615,0.039975
3,5,1,None,entropy,0.341311,0.052430
4,5,2,balanced,gini,0.359072,0.026980


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [21]:
params = {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50}
model = RandomForestClassifier(**params)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.9289940828402367
